# Humour Rating Generation

Run in order unless specified otherwise in Markdown.

In [ ]:
_COCKAMAMIE_DATA_SRC = "data/cockamamie_gobbledegook_us_data.json"
_DATAMUSE_DATA_SRC = "data/ol_gte2.2022-09-26.words"
_EH_DATA_SRC = "data/humor_dataset.csv"

_VEC_SRC = "data/wiki-news-300d-1M-subword.vec"
_PKL_SRC = "data/wiki-news-300d-1M-subword.pkl"

_OUTPUT_SRC = "output.json"

_COMPONENT_KEYS = ["snd", "scatc", "clq", "inslt", "juxt", "sexc"]


In [ ]:
import json
import numpy as np
import pickle as pkl

from collections import defaultdict
from gensim.models import KeyedVectors
from sklearn.linear_model import LinearRegression


Run **only one** of the two next blocks.

Run the 1st block to find humour scores for Cockamamie Gobbledegook words.

Run the 2nd block to find humour scores for Datamuse words (recommended).

In [ ]:
"""
Load sample_words from Cockamamie Gobbledegook data.
"""

with open(_COCKAMAMIE_DATA_SRC, "r") as f:
    user_data = json.load(f)
    votes = user_data["word_ratings"]["votes"]

sample_words = [sorted(ws) for ws in votes][0]  # 0th index for all 128k words


In [ ]:
"""
Load sample_words from Datamuse data.
"""

with open(_DATAMUSE_DATA_SRC, "r") as f:
    sample_words = f.read().splitlines()


In [ ]:
"""
Load training data for overall score from the EH dataset.
"""

with open(_EH_DATA_SRC, "r") as f:
    cells = [line.strip().split(",") for line in f.readlines()]
    headings = cells[0][1:]
    entries = {
        row[0]: {feat: float(v) for v, feat in zip(row[1:], headings)}
        for row in cells[1:]
    }

overall_training_data = {w: entries[w]["mean"] for w in entries}


In [ ]:
"""
Load training data for component scores from the Cockamamie Gobbledegook dataset.
"""

with open(_COCKAMAMIE_DATA_SRC, "r") as f:
    component_training_data = json.load(f)["word_features"]


In [ ]:
testing_words = list(overall_training_data) + list(component_training_data["snd"]) + sample_words  # combine all seen words
testing_words = list(set(testing_words))  # remove duplicates


Run the "Pickle the `.vec` file" section to create/update the `.pkl` file.

In [ ]:
"""
Predict the humour score of all words.
Outputs (word, humour score) pairs (.json) and ordered list (by humour) (.txt).
"""

def append_to_output_data(data, training_data, output_key):
    with open(_PKL_SRC, "rb") as f:
        E = pkl.load(f)

    clr = LinearRegression()
    clr.fit([E[w] for w in training_data], [training_data[w] for w in training_data])
    predictions = {
        w: float(v)
        for (w, v) in zip(testing_words, clr.predict([E[w] for w in testing_words]))
    }
    humour_order = [
        w for w in sorted(testing_words, key=lambda x: predictions[x], reverse=True)
    ]

    for word, score in predictions.items():
        data[word][output_key] = score
    
    return data

def export_output_data(data):
    with open(_OUTPUT_SRC, "w") as f:
        print(json.dumps(data), file=f)


In [ ]:
output_data = defaultdict(lambda: dict())

output_data = append_to_output_data(output_data, overall_training_data, "overall")
for key in _COMPONENT_KEYS:
    output_data = append_to_output_data(output_data, component_training_data[key], key)

export_output_data(output_data)


### Pickle the `.vec` file

In [ ]:
def vec_file2dict(vec_filename, pkl_filename):

    e = KeyedVectors.load_word2vec_format(vec_filename)

    def get_word(w):
        try:
            v = e[w.replace("_", " ")]
            return v / np.linalg.norm(v)  # normalize v
        except:
            return 0.0 * e["cat"]

    e_dict = {w: get_word(w) for w in testing_words}
    with open(pkl_filename, "wb") as f:
        pkl.dump(e_dict, f)

    non_zero_vectors = [v for v in e_dict.values() if not np.allclose(v, 0)]
    print(
        f'Loaded "{vec_filename}" and wrote {len(non_zero_vectors):,} non-zero vectors to "{pkl_filename}"'
    )


In [ ]:
vec_file2dict(_VEC_SRC, _PKL_SRC)
